In [1]:
import numpy
import pandas
import random
import sklearn
import lightgbm

In [2]:
# 求职者
person = pandas.read_csv("../trainset/person.csv", header=0,
                         names=["求职者编号", "性別", "工作年限", "最高学历", "应聘者专业", "年龄", "最近工作岗位", "最近所在行业", "当前工作所在地", "语言能力",
                                "专业特长"], encoding='gb18030')
person_sex = (person.性別 == "女").astype("float")
person_edu = person.最高学历.map({"其它": 0, "中专": 1, "高中（职高、技校）": 2, "大专": 3, "大学本科": 4, "硕士研究生": 5, "博士研究生": 6, "博士后": 7})
person_major = person.应聘者专业.astype("category")


In [365]:
person

,求职者编号,性別,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,语言能力,专业特长
0,33291,男,15,大专,计算机应用技术,37,网络管理/信息安全管理,NaN,深圳市,普通话：精通 广东话：精通******语：听：良好，说：良好，读：良好，写：,"1、精通计算机软、硬件及网络维护，能迅速处理各种突发的计算机故障 2、熟悉局域网建设和管理,..."
1,2985277,男,12,大专,计算机应用技术,35,*公关/营销/业务类,文化体育行业,深圳市,普通话：精通 广东话：精通************语：听：精通，说：精通，读,..
2,2982066,女,10,大专,金融学（含保险学）,32,出纳,医药销售行业,南山区,普通话：精通 广东话：精通************语：听：精通，说：良好，读,****-****年获校三等奖学金****-****年获单项奖学金
3,3010866,男,10,中专,物理电子学,34,营销代表/销售顾问,珠宝玉石行业,深圳市,普通话：精通 广东话：良好************语：听：一般，说：不会，读,"我对塑胶厂里的运作流程非常熟悉,从业务,计划,生产,人事,采购,客户服务都有经验."
4,316816964,女,15,中专,学前教育学,34,小学教育/幼儿教育/保育,行业组织,福田区,NaN,1：拥有丰富的办公室工作经历，能独挡一面处理工作中的相关事情。2：熟练掌握Windows办公...
...,...,...,...,...,...,...,...,...,...,...,...
54774,4546928,男,14,大专,NaN,47,产品开发,娱乐业,深圳市,NaN,★经验描述： 曾在所述行业或职位上成功开发过银行柜员机、自动售货机、医疗设备、双电源开关、塑...
54775,6261517,男,26,大学本科,财务管理,48,财务总监CFO/总会计师,香料香精行业,广州市,文案及沟通能力较好，有较强的人际穿透能力。,一、在财务会计工作方面：1．能熟练处理工商企业全盘账税，熟练操作用友、金蝶、江苏速达等财务软...
54776,5642172,男,13,大专,NaN,32,项目管理,银行业,深圳市,NaN,1.熟练使用PRO-E/CAD、UG、OFFICE软件；2.对模具结构及成型工艺有很深的理解...
54777,5433634,男,13,博士研究生,管理科学与工程,39,总裁/总经理/CEO/主席/社长/总编,塑胶行业,深圳市,"具有较强的语听说读写能力。可流利阅读文资料,并能熟练收发文邮件",****公司化企业财务总监工作经验；精通现代企业财务管理并具有深厚的财务管理理论与实践经验；...


In [3]:
# 求职意向
intent_table = pandas.read_csv("../trainset/person_cv.csv", header=0,
                               names=["求职者编号", "自荐信", "岗位类别", "工作地点", "所在行业", "可到职天数", "其他说明"])
intent_table["自荐信字数"] = intent_table.自荐信.str.len()

In [4]:
#工作经历
work_history = pandas.read_csv("../trainset/person_job_hist.csv", header=0,
                               names=["求职者编号", "岗位类别", "单位所在地", "单位所属行业", "主要业绩"])
work_history["主要业绩字数"] = work_history.主要业绩.str.len()

In [5]:
# 证书
cert_table = pandas.read_csv("../trainset/person_pro_cert.csv", header=0, names=["求职者编号", "专业证书名称", "备注"])

In [6]:
# 项目经历
project_history = pandas.read_csv("../trainset/person_project.csv", header=0,
                                  names=["求职者编号", "项目名称", "项目说明", "职责说明", "关键技术"])

In [7]:
# 岗位表
job_table = pandas.read_csv("../trainset/recruit.csv", header=0,
                            names=["岗位编号", "招聘对象代码", "招聘对象", "招聘职位", "对应聘者的专业要求", "岗位最低学历", "岗位工作地点", "岗位工作年限", "具体要求"])
job_table.招聘对象代码 = job_table.招聘对象代码.fillna(-1).astype("category")
job_table.招聘对象 = job_table.招聘对象.astype("category")
job_table.岗位最低学历 = job_table.岗位最低学历.map(
    {"其它": 0, "中专": 1, "高中（职高、技校）": 2, "大专": 3, "大学本科": 4, "硕士研究生": 5, "博士研究生": 6, "博士后": 7})
job_table.岗位工作年限 = job_table.岗位工作年限.map({"不限": -1, "应届毕业生": 0, "0至1年": 0, "1至2年": 1, "3至5年": 3, "5年以上": 5})
job_table["具体要求字数"] = job_table.具体要求.str.len()


In [8]:
job_table

,岗位编号,招聘对象代码,招聘对象,招聘职位,对应聘者的专业要求,岗位最低学历,岗位工作地点,岗位工作年限,具体要求,具体要求字数
0,135144,-1.0,NaN,业务员,NaN,2.0,深圳市,0,NaN,NaN
1,137045,-1.0,NaN,电子技术支持工程师,电子信息工程学,1.0,龙岗区,0,要求电子类中专以上文凭，为人诚恳，工作负责，熟悉PROTEL****自动排版系统；对计算机常...,70.0
2,146798,-1.0,NaN,仓管,【工商管理】,1.0,龙岗区,0,电子厂工作经验2年以上（非电子厂勿扰！）；十分熟悉仓库管理流程；懂金碟登帐者优先!,41.0
3,436321,2.0,社会无职,销售代表,NaN,1.0,深圳市,0,女，年龄****-****，高中以上学历，普通话粤语流利，善于沟通，有销售经验者优先。,43.0
4,440725,99.0,不限,造价员,工民建,1.0,深圳市,3,所学专业可以是其他专业，但应有工业与民用建筑土建工程、或给排水工程、或强弱电工程、或通风与空...,109.0
...,...,...,...,...,...,...,...,...,...,...
2099,44263687,1.0,社会在职,跟单文员,NaN,2.0,深圳市,1,女 1名 高中或以上文化，有两年以上丝移印跟单工作经验，字迹端正秀丽，责任心强。,42.0
2100,44274688,3.0,应届毕业,英语播音员,英语,3.0,宝安区,0,男女不限，英语本科以上学历，已过专业英语八级或者普通话二甲，口语好，音质甜美，善于模仿，有播...,66.0
2101,44285651,1.0,社会在职,电气工程师,NaN,NaN,深圳市,5,NaN,NaN
2102,44298679,2.0,社会无职,仓库管理员,NaN,2.0,深圳市,0,1. 1年以上电子厂仓库管理经验2. 熟练金碟登帐3. 能吃苦耐劳。4. 限女性,40.0


In [9]:
work_history_table = work_history.groupby("求职者编号").aggregate({"岗位类别": "count", "主要业绩字数": ["mean", "sum"]}).reset_index()
work_history_table.columns = ["求职者编号", "工作经历数", "平均主要业绩字数", "总主要业绩字数"]
project_history_table = project_history.groupby("求职者编号").aggregate({"项目名称": "count"}).reset_index()
project_history_table.columns = ["求职者编号", "项目经验数"]

In [10]:
work_history

,求职者编号,岗位类别,单位所在地,单位所属行业,主要业绩,主要业绩字数
0,1281276,行政管理,罗湖区,其它,1.协助总经理处理好日常事务及和外部公共关系；2.负责协助起草总经理各类工作往来文件，并负责...,158.0
1,980158,售前/售后服务,NaN,NaN,"本人主要负责万佳,天虹,岁宝,民润等重要客户的品牌分类管理,收集竞争对手信息与反馈,店内执行...",80.0
2,3016108,培训管理,福田区,信息行业（IT/通讯/互联网）,从事学生管理工作.并负责分校区的学生心理辅导和职业指导工作.,32.0
3,3016108,培训管理,福田区,信息行业（IT/通讯/互联网）,从事心理学的教学工作，并担任学校的心理辅导老师，负责了学校心理咨询中心的组建和日常咨...,79.0
4,2980989,产品开发,宝安区,电子行业,"1.工程师对产品进行设计及开发 2.处理3D图和2D图,同时制作相关的资料(如BOM\技术文...",93.0
...,...,...,...,...,...,...
115345,6198806,营销管理,深圳市,建筑工程业,公司从事城市夜景照明泛光亮化方面的工程设****公司主要客户群体为市政工程、房地产开发企业、...,223.0
115346,5863441,运营管理,NaN,信息行业（IT/通讯/互联网）,国内领先的微信开发服务商。基于微信为企业提供开发、运营、培训、推广一体化解决方案，帮助企业实...,563.0
115347,1408390,销售管理,深圳市,服装行业,****公司成****公司的销售，运营，招聘，采购，期****公司的采购和店铺销售和人员管理,46.0
115348,834069,人力资源管理,深圳市,珠宝玉石行业,在职期间任行政总监 及hrbp,15.0


In [11]:
work_history#[work_history.求职者编号==488]

,求职者编号,岗位类别,单位所在地,单位所属行业,主要业绩,主要业绩字数
0,1281276,行政管理,罗湖区,其它,1.协助总经理处理好日常事务及和外部公共关系；2.负责协助起草总经理各类工作往来文件，并负责...,158.0
1,980158,售前/售后服务,NaN,NaN,"本人主要负责万佳,天虹,岁宝,民润等重要客户的品牌分类管理,收集竞争对手信息与反馈,店内执行...",80.0
2,3016108,培训管理,福田区,信息行业（IT/通讯/互联网）,从事学生管理工作.并负责分校区的学生心理辅导和职业指导工作.,32.0
3,3016108,培训管理,福田区,信息行业（IT/通讯/互联网）,从事心理学的教学工作，并担任学校的心理辅导老师，负责了学校心理咨询中心的组建和日常咨...,79.0
4,2980989,产品开发,宝安区,电子行业,"1.工程师对产品进行设计及开发 2.处理3D图和2D图,同时制作相关的资料(如BOM\技术文...",93.0
...,...,...,...,...,...,...
115345,6198806,营销管理,深圳市,建筑工程业,公司从事城市夜景照明泛光亮化方面的工程设****公司主要客户群体为市政工程、房地产开发企业、...,223.0
115346,5863441,运营管理,NaN,信息行业（IT/通讯/互联网）,国内领先的微信开发服务商。基于微信为企业提供开发、运营、培训、推广一体化解决方案，帮助企业实...,563.0
115347,1408390,销售管理,深圳市,服装行业,****公司成****公司的销售，运营，招聘，采购，期****公司的采购和店铺销售和人员管理,46.0
115348,834069,人力资源管理,深圳市,珠宝玉石行业,在职期间任行政总监 及hrbp,15.0


In [12]:
project_history_table[project_history_table.求职者编号==6021388]

,求职者编号,项目经验数
849,6021388,9


In [297]:
# 待匹配标签
train_data = pandas.read_csv("../data/train_data.csv")
test_data = pandas.read_csv("../data/test_data.csv")
valid_data = pandas.read_csv("../data/valid_data.csv")

In [298]:
# train_data = pandas.read_csv("../data/trainset/recruit_folder.csv", header=0, names=["岗位编号", "求职者编号", "标签"])
# test_data = pandas.read_csv("../data/testset/recruit_folder.csv", header=0, names=["岗位编号", "求职者编号", "标签"])

# train,valid=train_test_split(train_data,test_size=0.2)

# train.reset_index(drop=True).to_csv(r'../data/train_data.csv')
# valid.reset_index(drop=True).to_csv(r'../data/valid_data.csv')
# test_data.reset_index(drop=True).to_csv(r'../data/test_data.csv')


In [299]:
data_all = pandas.concat([ train_data,valid_data,test_data], ignore_index=True)
candidate_data = data_all.groupby("求职者编号").aggregate({"岗位编号": "count"}).reset_index()
candidate_data.columns = ["求职者编号", "求职者求职数"]
job_data = data_all.groupby("岗位编号").aggregate({"求职者编号": "count"}).reset_index()
job_data.columns = ["岗位编号", "岗位求职者数"]

In [300]:
data_all.shape

(106065, 4)

In [301]:
candidate_data.sort_values(by='求职者求职数')

,求职者编号,求职者求职数
0,488,1
32885,6129742,1
32886,6129750,1
32889,6129766,1
32890,6129768,1
...,...,...
34660,6151649,32
24019,6034853,34
11687,5566970,38
16363,5892295,44


In [302]:
def get_data(data,feature):
    candidate_feature = feature.groupby("求职者编号").aggregate({"标签": "mean"}).reset_index()
    candidate_feature.columns = ["求职者编号", "求职者平均被匹配数"]
    job_feature = feature.groupby("岗位编号").aggregate({"标签": "mean"}).reset_index()
    job_feature.columns = ["岗位编号", "岗位平均被匹配数"]

    data = data.merge(person, on="求职者编号", how="left")
    data = data.merge(intent_table, on="求职者编号", how="left")
    data = data.merge(job_table, on="岗位编号", how="left")
    data = data.merge(project_history_table, on="求职者编号", how="left")
    data = data.merge(work_history_table, on="求职者编号", how="left")
    data = data.merge(candidate_data, on="求职者编号", how="left")
    data = data.merge(job_data, on="岗位编号", how="left")
    data = data.merge(candidate_feature, on="求职者编号", how="left")
    data = data.merge(job_feature, on="岗位编号", how="left")
    data["工作地点符合否"] = (data.工作地点 == data.岗位工作地点).astype("float")
#     print(data.columns)
    data_feature = data[["岗位编号", "求职者编号", "标签", "性別", "工作年限", "最高学历", "应聘者专业", "年龄", "自荐信字数", "可到职天数"
        , "项目经验数", "工作经历数", "平均主要业绩字数", "总主要业绩字数", "招聘对象代码", "招聘对象", "岗位最低学历", "岗位工作年限", "具体要求字数", "工作地点符合否"
         ,"求职者求职数", "岗位求职者数", "求职者平均被匹配数", "岗位平均被匹配数",
                             ]]

    data_feature = data_feature[["岗位编号", "求职者编号", "标签"] + [column for column in data_feature.columns if
                                                                 column not in ["岗位编号", "求职者编号", "标签"]]]

    return data_feature

In [303]:
data

,岗位编号,求职者编号,标签
0,825081,6256839,0
1,772899,5413605,0
2,795668,5219796,0
3,769754,5700693,0
4,773645,6208645,0
...,...,...,...
33874,826401,6068710,1
33875,832680,6276679,0
33876,805853,316210020,1
33877,792036,5712168,0


In [317]:
data_all.标签.value_counts()

0    100444
1      5621
Name: 标签, dtype: int64

In [319]:
data_all = pandas.concat([ train_data,valid_data,test_data], ignore_index=True)
k = 5
train = None
for i in range(k):
    data_label = train_data[train_data.index % k == i].reset_index(drop=True)
    data_feature = train_data[train_data.index % k != i].reset_index(drop=True)

    data_table = get_data(data_label, data_feature)
    train = pandas.concat([train, data_table], ignore_index=True)
valid = get_data(valid_data, train)
test = get_data(test_data, train)

In [322]:
train

,岗位编号,求职者编号,标签,性別,工作年限,最高学历,应聘者专业,年龄,自荐信字数,可到职天数,...,招聘对象代码,招聘对象,岗位最低学历,岗位工作年限,具体要求字数,工作地点符合否,求职者求职数,岗位求职者数,求职者平均被匹配数,岗位平均被匹配数
0,775040,5969825,0,男,12,大专,NaN,34,50.0,7.0,...,2.0,社会无职,2.0,0,46.0,1.0,8,88,0.0,0.000000
1,799873,6139430,0,男,12,大学本科,机电一体化,37,148.0,7.0,...,-1.0,NaN,3.0,3,99.0,1.0,1,214,NaN,0.000000
2,779698,6083324,0,男,0,中专,特殊教育学,31,65.0,7.0,...,1.0,社会在职,1.0,1,71.0,0.0,1,593,NaN,0.000000
3,771109,5873085,0,女,6,大学本科,财务管理,33,39.0,7.0,...,-1.0,NaN,2.0,1,37.0,0.0,2,273,NaN,0.000000
4,823833,6218282,0,女,0,大专,英语,29,NaN,7.0,...,-1.0,NaN,2.0,0,57.0,0.0,6,303,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28227,800360,1633998,0,女,18,大专,金融学（含保险学）,40,137.0,7.0,...,-1.0,NaN,1.0,0,38.0,0.0,8,85,1.0,0.200000
28228,43922680,5733578,1,女,10,高中（职高、技校）,计算机应用技术,28,119.0,14.0,...,1.0,社会在职,NaN,5,NaN,0.0,2,233,NaN,1.000000
28229,816862,6088168,0,男,0,大学本科,NaN,30,149.0,7.0,...,2.0,社会无职,2.0,0,41.0,0.0,1,246,NaN,0.017857
28230,772339,4684178,0,男,11,大专,NaN,34,76.0,7.0,...,2.0,社会无职,NaN,0,23.0,0.0,7,290,NaN,0.000000


In [321]:
train_all.shape,train_data.shape,valid_data.shape

((35291, 24), (28232, 4), (7059, 4))

In [323]:
temp_data= pandas.concat([train,valid, test], ignore_index=True)

In [348]:
data_all_feature = pandas.get_dummies(temp_data[['性別','工作年限','最高学历','应聘者专业','可到职天数','招聘对象代码', '招聘对象',
       '岗位最低学历','工作经历数','岗位工作年限','工作地点符合否',"求职者平均被匹配数", "岗位平均被匹配数", "求职者求职数", "岗位求职者数"]])#[['性别']]#[']性别']]
data_all_feature['年龄']=temp_data['年龄']
data_all_feature['项目经验数']=temp_data['项目经验数']
data_all_feature['具体要求字数']=temp_data['具体要求字数']
data_all_feature['自荐信字数'] = temp_data['自荐信字数']
data_all_feature['平均主要业绩字数'] =temp_data['平均主要业绩字数']
data_all_feature['总主要业绩字数']=temp_data['总主要业绩字数']
data_all_feature['岗位编号']=temp_data['岗位编号']
data_all_feature['求职者编号']=temp_data['求职者编号']
data_all_feature['标签']=temp_data['标签']

In [352]:
train = data_all_feature[:train.shape[0]]#.标签.value_counts()
valid = data_all_feature[train.shape[0]:train.shape[0]+valid.shape[0]]
test = data_all_feature[train.shape[0]+valid.shape[0]:]
train_label = train[train.columns[-1]]
valid_label = valid[valid.columns[-1]]
train=train[train.columns[:-3]]
valid=valid[valid.columns[:-3]]
test=test[test.columns[:-3]]


In [353]:
train

,工作年限,可到职天数,岗位最低学历,工作经历数,岗位工作年限,工作地点符合否,求职者平均被匹配数,岗位平均被匹配数,求职者求职数,岗位求职者数,...,招聘对象_不限,招聘对象_应届毕业,招聘对象_社会在职,招聘对象_社会无职,年龄,项目经验数,具体要求字数,自荐信字数,平均主要业绩字数,总主要业绩字数
0,12,7.0,2.0,4.0,0,1.0,0.0,0.000000,8,88,...,0,0,0,1,34,NaN,46.0,50.0,161.250000,645.0
1,12,7.0,3.0,2.0,3,1.0,NaN,0.000000,1,214,...,0,0,0,0,37,NaN,99.0,148.0,115.500000,231.0
2,0,7.0,1.0,2.0,1,0.0,NaN,0.000000,1,593,...,0,0,1,0,31,NaN,71.0,65.0,109.000000,218.0
3,6,7.0,2.0,1.0,1,0.0,NaN,0.000000,2,273,...,0,0,0,0,33,NaN,37.0,39.0,77.000000,77.0
4,0,7.0,2.0,3.0,0,0.0,0.0,0.000000,6,303,...,0,0,0,0,29,NaN,57.0,NaN,81.333333,244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28227,18,7.0,1.0,4.0,0,0.0,1.0,0.200000,8,85,...,0,0,0,0,40,NaN,38.0,137.0,118.500000,474.0
28228,10,14.0,NaN,3.0,5,0.0,NaN,1.000000,2,233,...,0,0,1,0,28,NaN,NaN,119.0,77.000000,154.0
28229,0,7.0,2.0,4.0,0,0.0,NaN,0.017857,1,246,...,0,0,0,1,30,NaN,41.0,149.0,117.750000,471.0
28230,11,7.0,NaN,2.0,0,0.0,NaN,0.000000,7,290,...,0,0,0,1,34,NaN,23.0,76.0,420.500000,841.0


In [354]:
train.shape,valid.shape,test.shape

((28232, 454), (7059, 454), (70774, 454))

In [355]:
train

,工作年限,可到职天数,岗位最低学历,工作经历数,岗位工作年限,工作地点符合否,求职者平均被匹配数,岗位平均被匹配数,求职者求职数,岗位求职者数,...,招聘对象_不限,招聘对象_应届毕业,招聘对象_社会在职,招聘对象_社会无职,年龄,项目经验数,具体要求字数,自荐信字数,平均主要业绩字数,总主要业绩字数
0,12,7.0,2.0,4.0,0,1.0,0.0,0.000000,8,88,...,0,0,0,1,34,NaN,46.0,50.0,161.250000,645.0
1,12,7.0,3.0,2.0,3,1.0,NaN,0.000000,1,214,...,0,0,0,0,37,NaN,99.0,148.0,115.500000,231.0
2,0,7.0,1.0,2.0,1,0.0,NaN,0.000000,1,593,...,0,0,1,0,31,NaN,71.0,65.0,109.000000,218.0
3,6,7.0,2.0,1.0,1,0.0,NaN,0.000000,2,273,...,0,0,0,0,33,NaN,37.0,39.0,77.000000,77.0
4,0,7.0,2.0,3.0,0,0.0,0.0,0.000000,6,303,...,0,0,0,0,29,NaN,57.0,NaN,81.333333,244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28227,18,7.0,1.0,4.0,0,0.0,1.0,0.200000,8,85,...,0,0,0,0,40,NaN,38.0,137.0,118.500000,474.0
28228,10,14.0,NaN,3.0,5,0.0,NaN,1.000000,2,233,...,0,0,1,0,28,NaN,NaN,119.0,77.000000,154.0
28229,0,7.0,2.0,4.0,0,0.0,NaN,0.017857,1,246,...,0,0,0,1,30,NaN,41.0,149.0,117.750000,471.0
28230,11,7.0,NaN,2.0,0,0.0,NaN,0.000000,7,290,...,0,0,0,1,34,NaN,23.0,76.0,420.500000,841.0


In [356]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

# train_feature,test_feature,train_label,test_label =train_test_split(data_feature[:train.shape[0]].values,train["标签"],test_size =0.2)
train_set=lightgbm.Dataset(train.values, label=train_label)
val_set=lightgbm.Dataset(valid.values, label=valid_label)

model = lightgbm.train(train_set=train_set,
                       valid_sets=[val_set],
                       num_boost_round=3000,
                       
                       params={"objective": "binary", "learning_rate": 0.03, "max_depth": 6, "num_leaves": 32,
                               "verbose": -1, "bagging_fraction": 0.8, "feature_fraction": 0.8},
                       feval=lgb_f1_score,
                      early_stopping_rounds=100)


[1]	valid_0's binary_logloss: 0.429279	valid_0's f1: 0
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.409421	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.392373	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.377027	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.363244	valid_0's f1: 0
[6]	valid_0's binary_logloss: 0.35088	valid_0's f1: 0
[7]	valid_0's binary_logloss: 0.3455	valid_0's f1: 0
[8]	valid_0's binary_logloss: 0.339504	valid_0's f1: 0
[9]	valid_0's binary_logloss: 0.333782	valid_0's f1: 0
[10]	valid_0's binary_logloss: 0.323634	valid_0's f1: 0
[11]	valid_0's binary_logloss: 0.31423	valid_0's f1: 0
[12]	valid_0's binary_logloss: 0.30555	valid_0's f1: 0
[13]	valid_0's binary_logloss: 0.297323	valid_0's f1: 0
[14]	valid_0's binary_logloss: 0.29326	valid_0's f1: 0
[15]	valid_0's binary_logloss: 0.285803	valid_0's f1: 0
[16]	valid_0's binary_logloss: 0.278928	valid_0's f1: 0.0279965
[17]	valid_0's binary_logloss: 0.272355	valid_0's

[135]	valid_0's binary_logloss: 0.1272	valid_0's f1: 0.849929
[136]	valid_0's binary_logloss: 0.127104	valid_0's f1: 0.849524
[137]	valid_0's binary_logloss: 0.127001	valid_0's f1: 0.849929
[138]	valid_0's binary_logloss: 0.126875	valid_0's f1: 0.849929
[139]	valid_0's binary_logloss: 0.126778	valid_0's f1: 0.849929
[140]	valid_0's binary_logloss: 0.126689	valid_0's f1: 0.849929
[141]	valid_0's binary_logloss: 0.126566	valid_0's f1: 0.850476
[142]	valid_0's binary_logloss: 0.126446	valid_0's f1: 0.850476
[143]	valid_0's binary_logloss: 0.12635	valid_0's f1: 0.850071
[144]	valid_0's binary_logloss: 0.12628	valid_0's f1: 0.849667
[145]	valid_0's binary_logloss: 0.126204	valid_0's f1: 0.849667
[146]	valid_0's binary_logloss: 0.126116	valid_0's f1: 0.849667
[147]	valid_0's binary_logloss: 0.126054	valid_0's f1: 0.849667
[148]	valid_0's binary_logloss: 0.125999	valid_0's f1: 0.850214
[149]	valid_0's binary_logloss: 0.125905	valid_0's f1: 0.850214
[150]	valid_0's binary_logloss: 0.125794	val

In [357]:
model.feature_importance()

array([399, 111, 111,  64,  67,  41, 259, 736, 370, 540, 255,  66,   8,
         0,   0,   0,  23,  16,   3,  22,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   6,   0,   0,   0,   0,   0,   3,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   8,   0,   0,   0,   0,   0,   0,   6,
         0,   0,   0,   0,   0,   0,   3,  11,   0,   0,   0,   0,   8,
         0,   0,   0,   0,   0,   0,  10,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  12,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   1,  11,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  15,   

In [359]:
dict(zip(list(train.columns),model.feature_importance()))

{'工作年限': 399,
 '可到职天数': 111,
 '岗位最低学历': 111,
 '工作经历数': 64,
 '岗位工作年限': 67,
 '工作地点符合否': 41,
 '求职者平均被匹配数': 259,
 '岗位平均被匹配数': 736,
 '求职者求职数': 370,
 '岗位求职者数': 540,
 '性別_女': 255,
 '性別_男': 66,
 '最高学历_中专': 8,
 '最高学历_其它': 0,
 '最高学历_博士后': 0,
 '最高学历_博士研究生': 0,
 '最高学历_大专': 23,
 '最高学历_大学本科': 16,
 '最高学历_硕士研究生': 3,
 '最高学历_高中（职高、技校）': 22,
 '应聘者专业_【中医学】': 0,
 '应聘者专业_【中国语言文学】': 0,
 '应聘者专业_【临床医学】': 0,
 '应聘者专业_【交通运输工程】': 0,
 '应聘者专业_【仪器科学与技术】': 0,
 '应聘者专业_【体育学】': 0,
 '应聘者专业_【信息与通信工程】': 0,
 '应聘者专业_【公共卫生与预防医学】': 0,
 '应聘者专业_【公共管理】': 0,
 '应聘者专业_【公安学】': 0,
 '应聘者专业_【其他专业】': 6,
 '应聘者专业_【军事学】': 0,
 '应聘者专业_【农业工程】': 0,
 '应聘者专业_【农学】': 0,
 '应聘者专业_【力学】': 0,
 '应聘者专业_【动力工程及工程热物理】': 0,
 '应聘者专业_【化学】': 3,
 '应聘者专业_【化学工程】': 0,
 '应聘者专业_【历史学】': 0,
 '应聘者专业_【口腔医学】': 0,
 '应聘者专业_【哲学】': 0,
 '应聘者专业_【园艺学】': 0,
 '应聘者专业_【土木工程】': 0,
 '应聘者专业_【地理学】': 0,
 '应聘者专业_【地质学】': 0,
 '应聘者专业_【基础医学】': 0,
 '应聘者专业_【外语类】': 0,
 '应聘者专业_【大气科学】': 0,
 '应聘者专业_【天文学】': 0,
 '应聘者专业_【工商管理】': 0,
 '应聘者专业_【建筑学】': 0,
 '应聘者专业_【心理学】': 0,
 '应聘者专业_【情报与档案管理】': 0,
 '应聘者专业_【政治

In [360]:
test_data["预测打分"] = model.predict(test.values)
test_data = test_data.sort_values("预测打分", ascending=False, ignore_index=True)
test_data["预测"] = 0


In [361]:
test_data.loc[:int(0.15 * len(test)), ["预测"]] = 1

submit = test_data [["岗位编号", "求职者编号", "预测"]]
submit.columns = ["RECRUIT_ID", "PERSON_ID", "LABEL"]
submit.to_csv("result_v4.csv", index=False)

In [219]:
test = test.sort_values("预测打分", ascending=False, ignore_index=True)
test["预测"] = 0

In [220]:
test.loc[:int(0.15 * len(test)), ["预测"]]

,预测
0,0
1,0
2,0
3,0
4,0
...,...
10612,0
10613,0
10614,0
10615,0


In [221]:
test.loc[:int(0.15 * len(test)), ["预测"]] = 1

In [ ]:
# 0.8521